<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=9a7db0109bcde97789f4d6111c23b29ce4b1dd4becc7458aca5ccfb81b41b1ec
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.3
    Uninstalling protobuf-6.33.3:
      Successfully uninstalled protobuf-6.33.3


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

  Attempting uninstall: yfinance
    Found existing installation: yfinance 1.0
    Uninstalling yfinance-1.0:
      Successfully uninstalled yfinance-1.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2026-01-12 13:15:39
-------------------
qualified stocks: 86
with latest results: 25
still star stocks: 14
-------------------
Initial Investment:  1.30 C
CY Investment:  1.57 C
Reserve:  1.68 K
Current:  1.40 C
-------------------
Today PnL: -59.64 K (-0.42%)
Current PnL: -28.62 L (-18.26%)
CY Booked + Current PnL: -14.33 L (-9.14%)
-------------------
Total profit:  1.72 L
Total loss:  -30.35 L
-------------------
Total Booked + Current PnL: 12.65 L (9.74%)
Total Booked PnL: 41.27 L (31.79%)
Curr Year Booked PnL: 14.29 L (10.17%)
Prev Year Booked PnL: 26.98 L (20.78%)
Est FTT:  2.35 C
Est FTT PnL: 94.08 L (66.98%)
Deployed:  1.30 C
Current:  1.40 C
CAGR/XIRR %: 4.26%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
51,NESTLEIND,1377.00,0.45,15.04,5.53,21.40,16886.0,39921.0,305347.0,6.71,62.0,X-LC,5.87,14.0,2.36,2.17,22.32,XY25,NTT,FMCG
17,COALINDIA,484.83,2.84,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.31,XY25,ATH,MINING
77,TTKPRESTIG,770.00,-1.98,-22.27,28.77,0.09,22536.0,-22446.0,78331.0,81.86,34.0,M-SC,3.34,253.0,-1.00,0.56,2.91,OX40N,NTT,DURABLES
49,MASFIN,398.61,0.35,-4.56,27.88,22.05,26071.0,-4470.0,93510.0,-17.80,45.0,H-SC,6.25,168.0,-0.17,0.67,35.92,XR,ATH,FINANCE
39,INDIGOPNTS,1408.00,-0.61,-15.25,18.02,0.02,26657.0,-26627.0,147932.0,125.45,51.0,M-SC,8.30,240.0,-1.00,1.05,28.68,OX40N,NTT,PAINTS


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,3.34,-2.97,53.17,48.63,103163.0,-5930.0,194024.0,-35.53,52.0,H-SC,13.72,137.0,-0.06,1.38,8.31,XR,NTT,MISC
17,COALINDIA,484.83,2.84,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.31,XY25,ATH,MINING
40,INDUSINDBK,1800.00,2.49,-33.25,99.07,32.89,51060.0,-25668.0,51539.0,-736.24,64.0,L-MC,7.91,259.0,-0.50,0.37,41.94,XR,NTT,BANKS
18,COFFEEDAY,80.00,1.99,-42.00,133.24,35.27,87747.0,-47693.0,65856.0,-55.73,45.0,L-SC,4.45,268.0,-0.54,0.47,61.18,XR,NTT,HOTELS
57,RECLTD,446.00,1.73,0.14,20.57,20.74,41849.0,275.0,203445.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.77,XY25,NTT,FINANCE


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
65,SIEMENS,4671.50,-3.35,-20.96,58.80,25.51,86489.0,-39005.0,147090.0,-9.63,34.0,H-LC,1.98,50.0,-0.45,1.05,8.39,AR,ATH,ELECTRICAL
55,RAJESHEXPO,518.00,-3.16,-68.44,217.11,0.07,93983.0,-93889.0,43288.0,1487.12,32.0,L-SC,2.07,269.0,-1.00,0.31,7.66,OX40N,NTT,JEWELLERY
76,TRIDENT,48.00,-3.06,-32.02,89.05,28.51,55846.0,-29542.0,62713.0,-19.62,32.0,M-SC,7.77,226.0,-0.53,0.45,7.31,XR,NTT,TEXTILES
60,REPCOHOME,880.00,-2.37,-11.93,115.53,89.82,291516.0,-34176.0,252329.0,-56.04,48.0,H-SC,4.23,161.0,-0.12,1.80,33.50,XY24,NTT,FINANCE
9,BAJAJHLDNG,14451.52,-2.27,-3.50,33.91,29.22,62213.0,-6661.0,183464.0,-6.00,37.0,X-LC,11.73,29.0,-0.11,1.31,5.36,X40,ATH,FINANCE


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.0,1.73,0.14,20.57,20.74,41849.0,275.0,203445.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.77,XY25,NTT,FINANCE


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-2.13,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,44.0,M-SC,10.11,220.0,-0.04,1.00,5.17,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.61,-15.25,18.02,0.02,26657.0,-26627.0,147932.0,125.45,51.0,M-SC,8.30,240.0,-1.00,1.05,28.68,OX40N,NTT,PAINTS
72,TATAELXSI,9161.00,-0.33,-18.26,61.51,32.03,66288.0,-24067.0,107768.0,-11.50,63.0,H-SC,6.83,157.0,-0.36,0.77,20.71,OX40N,NTT,IT
77,TTKPRESTIG,770.00,-1.98,-22.27,28.77,0.09,22536.0,-22446.0,78331.0,81.86,34.0,M-SC,3.34,253.0,-1.00,0.56,2.91,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-0.66,-22.93,47.24,13.47,98178.0,-61839.0,207828.0,-68.31,45.0,H-SC,4.51,159.0,-0.63,1.48,5.67,XY24,NTT,PAINTS


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
57,RECLTD,446.00,1.73,0.14,20.57,20.74,41849.0,275.0,203445.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.77,XY25,NTT,FINANCE
17,COALINDIA,484.83,2.84,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.31,XY25,ATH,MINING
84,WIPRO,420.00,0.71,8.20,59.21,72.27,108556.0,13893.0,183341.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.88,XR,NTT,IT


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
84,WIPRO,420.00,0.71,8.20,59.21,72.27,108556.0,13893.0,183341.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.88,XR,NTT,IT
25,FINCABLES,1641.55,-2.13,-4.47,117.64,107.91,165925.0,-6599.0,141045.0,-22.17,44.0,M-SC,10.11,220.0,-0.04,1.00,5.17,OX40N,ATH,CABLES
37,IEX,219.00,3.34,-2.97,53.17,48.63,103163.0,-5930.0,194024.0,-35.53,52.0,H-SC,13.72,137.0,-0.06,1.38,8.31,XR,NTT,MISC
38,INDIAMART,4810.62,-0.24,-8.03,124.77,106.72,141528.0,-9905.0,113431.0,-54.44,34.0,H-SC,7.66,138.0,-0.07,0.81,13.47,AR,ATH,MISC
85,ZYDUSLIFE,1286.17,-0.38,-5.99,44.27,35.63,87222.0,-12552.0,197022.0,-19.40,34.0,H-MC,3.35,119.0,-0.14,1.40,9.68,AR,ATH,PHARMA


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.30,-17.12,33.67,10.78,67061.0,-41148.0,199170.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.30,X40,NTT,FMCG
7,AWL,485.00,-1.26,-28.64,120.28,57.19,279916.0,-93407.0,232720.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.66,0.00,XY24,NTT,FMCG
52,PAGEIND,51605.08,-0.01,-13.25,50.56,30.62,103977.0,-31404.0,205650.0,-38.32,25.0,X-MC,16.67,57.0,-0.30,1.46,0.00,X40,ATH,APPARELS
11,BATAINDIA,2096.00,-1.85,-41.36,130.58,35.20,98519.0,-53223.0,75447.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
78,UNITDSPR,1644.00,-0.58,-2.79,24.23,20.77,55149.0,-6536.0,227608.0,-11.23,29.0,X-MC,1.32,70.0,-0.12,1.62,3.93,X40N,NTT,BREWERIES


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
78,UNITDSPR,1644.00,-0.58,-2.79,24.23,20.77,55149.0,-6536.0,227608.0,-11.23,29.0,X-MC,1.32,70.0,-0.12,1.62,3.93,X40N,NTT,BREWERIES
45,JIOFIN,387.00,0.23,-7.39,34.42,24.48,85222.0,-19771.0,247594.0,-8.97,39.0,X-LC,1.54,37.0,-0.23,1.76,43.51,XY24,BTT,FINANCE
65,SIEMENS,4671.50,-3.35,-20.96,58.80,25.51,86489.0,-39005.0,147090.0,-9.63,34.0,H-LC,1.98,50.0,-0.45,1.05,8.39,AR,ATH,ELECTRICAL
81,VBL,671.64,0.71,-0.53,36.39,35.67,114331.0,-1659.0,314183.0,-12.33,55.0,X-LC,2.28,5.0,-0.01,2.24,13.26,X40N,ATH,FMCG
73,TCS,4389.96,0.15,-11.67,36.65,20.71,122448.0,-44137.0,334100.0,-23.95,48.0,X-LC,2.33,3.0,-0.36,2.38,11.64,X40,ATH,IT


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
3,ACC,3906.00,-0.35,-28.67,130.13,64.15,220870.0,-68221.0,169730.0,-58.06,32.0,X-SC,6.40,84.0,-0.31,1.21,0.0,XY24,BTT,CEMENT
7,AWL,485.00,-1.26,-28.64,120.28,57.19,279916.0,-93407.0,232720.0,-66.31,21.0,X-MC,9.83,58.0,-0.33,1.66,0.0,XY24,NTT,FMCG
11,BATAINDIA,2096.00,-1.85,-41.36,130.58,35.20,98519.0,-53223.0,75447.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.0,X40,NTT,FOOTWEAR
53,PGHH,17907.65,-0.68,-6.75,43.91,34.19,87427.0,-14416.0,199104.0,-34.39,33.0,X-MC,4.08,60.0,-0.16,1.42,0.0,X40,ATH,FMCG
58,RELAXO,1176.00,-1.70,-49.04,198.86,52.31,147113.0,-71182.0,73978.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.0,X40N,NTT,FOOTWEAR


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
54,QUESS,424.00,-0.82,-28.78,100.58,42.84,46563.0,-18712.0,46294.0,-53.17,47.0,X-SC,26.53,83.0,-0.40,0.33,4.74,XY24,NTT,MISC
58,RELAXO,1176.00,-1.70,-49.04,198.86,52.31,147113.0,-71182.0,73978.0,-45.70,35.0,X-SC,7.94,92.0,-0.48,0.53,0.00,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.85,-41.36,130.58,35.20,98519.0,-53223.0,75447.0,-1.08,27.0,X-SC,17.48,93.0,-0.54,0.54,0.00,X40,NTT,FOOTWEAR
50,MEDANTA,1486.00,1.03,-0.85,25.14,24.08,31645.0,-1075.0,125875.0,-8.86,48.0,X-SC,6.16,91.0,-0.03,0.90,18.60,XY24,NTT,HEALTHCARE
34,HONAUT,58357.33,-0.66,-18.27,75.35,43.32,100306.0,-29752.0,133120.0,-29.79,41.0,X-SC,7.95,90.0,-0.30,0.95,2.21,X40N,ATH,ELECTRICAL


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.00,0.30,-17.12,33.67,10.78,67061.0,-41148.0,199170.0,-50.55,14.0,X-LC,7.09,1.0,-0.61,1.42,0.30,X40,NTT,FMCG
75,TMPV,600.00,-0.85,-28.03,70.87,22.98,115222.0,-63316.0,162582.0,-25.52,39.0,X-LC,4.09,2.0,-0.55,1.16,2.26,XY24,NTT,AUTO
73,TCS,4389.96,0.15,-11.67,36.65,20.71,122448.0,-44137.0,334100.0,-23.95,48.0,X-LC,2.33,3.0,-0.36,2.38,11.64,X40,ATH,IT
81,VBL,671.64,0.71,-0.53,36.39,35.67,114331.0,-1659.0,314183.0,-12.33,55.0,X-LC,2.28,5.0,-0.01,2.24,13.26,X40N,ATH,FMCG
41,INFY,1972.00,-0.71,9.10,23.04,34.24,78653.0,28487.0,341375.0,-14.93,47.0,X-LC,7.65,6.0,0.36,2.43,18.14,X40,NTT,IT


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,2.49,-33.25,99.07,32.89,51060.0,-25668.0,51539.0,-736.24,64.0,L-MC,7.91,259.0,-0.50,0.37,41.94,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,-0.61,-15.25,18.02,0.02,26657.0,-26627.0,147932.0,125.45,51.0,M-SC,8.30,240.0,-1.00,1.05,28.68,OX40N,NTT,PAINTS
17,COALINDIA,484.83,2.84,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.31,XY25,ATH,MINING
84,WIPRO,420.00,0.71,8.20,59.21,72.27,108556.0,13893.0,183341.0,-6.99,52.0,M-LC,6.21,101.0,0.13,1.31,14.88,XR,NTT,IT
82,VOLTAS,1856.00,0.22,15.00,26.26,45.20,57903.0,28758.0,220500.0,3.76,63.0,X-MC,5.20,80.0,0.50,1.57,22.82,XY25,NTT,AC


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
0,5PAISA,593.00,1.24,-30.74,63.02,12.90,95362.0,-67176.0,151320.0,122.36,58.0,H-SC,10.01,173.0,-0.70,1.08,25.82,OX40N,NTT,FINANCE
72,TATAELXSI,9161.00,-0.33,-18.26,61.51,32.03,66288.0,-24067.0,107768.0,-11.50,63.0,H-SC,6.83,157.0,-0.36,0.77,20.71,OX40N,NTT,IT
17,COALINDIA,484.83,2.84,5.80,12.69,19.23,20201.0,8732.0,159192.0,24.27,71.0,L-LC,7.65,182.0,0.43,1.13,28.31,XY25,ATH,MINING
57,RECLTD,446.00,1.73,0.14,20.57,20.74,41849.0,275.0,203445.0,31.15,56.0,M-MC,5.46,189.0,0.01,1.45,10.77,XY25,NTT,FINANCE
40,INDUSINDBK,1800.00,2.49,-33.25,99.07,32.89,51060.0,-25668.0,51539.0,-736.24,64.0,L-MC,7.91,259.0,-0.50,0.37,41.94,XR,NTT,BANKS


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,20.39
1,25,44.44
2,50,76.56


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    42.55
MC    30.77
LC    26.68
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     26.29
X40      24.01
X40N     12.79
XY25     10.84
XR        9.00
AR        8.51
OX40N     7.61
SR        0.95
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
X-MC    24.37
X-LC    23.19
H-SC    22.78
M-SC    11.19
X-SC     7.80
H-MC     4.58
M-MC     1.45
M-LC     1.31
L-LC     1.13
H-LC     1.05
L-SC     0.78
L-MC     0.37
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,17.17,-9.19,45.01
IT,12.92,-17.95,77.44
FINANCE,11.09,-14.65,62.77
MISC,7.05,-31.06,84.67
ELECTRICAL,5.87,-13.82,55.51
PAINTS,5.81,-13.20,30.01
INSURANCE,4.73,-1.87,36.72
PHARMA,3.95,-5.89,39.22
AUTO,2.78,-34.37,80.95


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3278048.0,21
AR,1376783.0,10
XR,1231203.0,12
X40,1147937.0,15
X40N,945673.0,9
OX40N,722803.0,10
XY25,417851.0,7
SR,287773.0,2


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3597479.0,24
X-MC,1608122.0,16
M-SC,1486244.0,15
X-LC,966488.0,13
X-SC,825564.0,8
H-MC,434289.0,3
L-SC,181730.0,2
M-LC,108556.0,1
H-LC,86489.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1270178.0      6
           AR         929312.0      5
M-SC       XY24       850428.0      6
H-SC       XR         795695.0      7
X-MC       X40        523531.0      7
           XY24       451241.0      3
X-LC       X40        424024.0      6
X-MC       X40N       409480.0      4
X-SC       X40N       326104.0      3
H-SC       OX40N      314521.0      4
M-SC       OX40N      314299.0      5
X-SC       XY24       299078.0      3
H-SC       SR         287773.0      2
H-MC       AR         227610.0      2
X-MC       XY25       223870.0      2
X-LC       X40N       210089.0      2
H-MC       XY24       206679.0      1
X-LC       XY24       200444.0      2
X-SC       X40        200382.0      2
M-SC       XR         188145.0      2
           AR         133372.0      2
X-LC       XY25       131931.0      3
M-LC       XR         108556.0      1
L-SC       OX40N       93983.0      1
           XR          87747.0      1
H-LC       AR          86489.0      1
L-MC       XR          51060.0      1
M-MC       XY25        41849.0      1
L-LC       XY25        20201.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
